# HCPCS-5: length of HCPCS is 5

Description: check if all HCPCS codes have a length of 5.

Starting Author: Amy Jin (amy@careset.com)

Date: May 2nd, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [2]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) col_name:               column to test
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, the test will print out all failed HCPCS codes.


def hcpcs_5(db_name, table_name, col_name):
    
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        print ('Test file: {}.{}'.format(db_name, table_name))
        print ('\n')
        
        query = ('''
            SELECT *
            FROM {db}.{t1}
            WHERE LENGTH({col1}) <> 5
            LIMIT 1;
        '''.format(db = db_name, t1 = table_name, col1 = col_name))

        cur.execute(query)
        rows = list(sum(cur.fetchall(), ()))

        
        if not len(rows):
            print ("Test result: PASS")
        else:
            print ("Test result: FAIL" + '\n')
            print ("The following {} fail the test:".format(col_name) + '\n')
            
            # MySQL query to get all failed HCPCS codes
            query = ('''
                SELECT DISTINCT {col1}
                FROM {db}.{t1}
                WHERE LENGTH({col1}) <> 5;
            '''.format(db = db_name, t1 = table_name, col1 = col_name))

            cur.execute(query)
            
            for row in cur.fetchall():
                for i in range(0,len(row)):
                    print (str(row[i]), end=", ")
                print ('\n')
            
        cur.close()
        connection.close()

## Test Example

In [3]:
hcpcs_5('hcpcs_global_carrier', 'hcpcs_global_carrier_16_17_6','hcpcs_cd')

Test file: hcpcs_global_carrier.hcpcs_global_carrier_16_17_6


Test result: PASS


In [4]:
hcpcs_5('_amy', 'test_data_good','hcpcs_cd')

Test file: _amy.test_data_good


Test result: FAIL

The following hcpcs_cd fail the test:

7003, 

0, 

1, 

2, 

4, 

6, 

7, 

20, 

42, 

78, 

91, 

100, 

102, 

103, 

104, 

120, 

124, 

126, 

140, 

142, 

144, 

145, 

147, 

148, 

150, 

160, 

162, 

164, 

170, 

172, 

174, 

176, 

180, 

190, 

192, 

199, 

210, 

211, 

212, 

213, 

214, 

215, 

216, 

218, 

220, 

222, 

230, 

258, 

283, 

300, 

320, 

322, 

326, 

330, 

350, 

352, 

366, 

400, 

402, 

404, 

406, 

410, 

450, 

454, 

470, 

472, 

474, 

478, 

500, 

508, 

520, 

522, 

524, 

528, 

529, 

530, 

532, 

534, 

537, 

539, 

540, 

541, 

542, 

546, 

548, 

550, 

560, 

561, 

562, 

563, 

566, 

567, 

580, 

599, 

600, 

604, 

620, 

625, 

626, 

630, 

632, 

635, 

640, 

670, 

700, 

702, 

730, 

740, 

750, 

752, 

754, 

756, 

770, 

790, 

792, 

794, 

796, 

797, 

800, 

802, 

810, 

820, 

830, 

832, 

834, 

836, 

840, 

842, 

844, 

846, 

848, 

851, 



In [4]:
hcpcs_5('_amy', 'test_data2','hcpcs_cd')

Test file: _amy.test_data2


Test result: FAIL

The following HCPCS(s) fail the test:

F0, 

, 

7321, 

8540, 

4387, 

9243, 

3320, 

9040, 

7285, 

7012, 

4341, 

4241, 

9167, 

9215, 

7140, 

6640, 

9310, 

8730, 

3927, 

7286, 

6840, 

9837, 

7431, 

7955, 

